In [1]:
from drowsiness_detection.data import load_raw_60_sec_data
import numpy as np
from drowsiness_detection import config
from joblib import dump
import time
from drowsiness_detection.models import build_cnn_model

In [2]:
data = load_raw_60_sec_data()

data.X_train = np.concatenate([data.X_train, data.X_val])
data.y_train = np.concatenate([data.y_train, data.y_val])


In [3]:
model_name = "cnn"

best_params = {
    "input_shape": (20, 1800, 7),
    "dropout_rate": 0.89,
    "kernel_size": 5,
    "learning_rate": 0.017,
    "num_conv_layers": 2,
    "num_filters": 124,
    "pooling": "average",
    "stride": 3
  }

model = build_cnn_model(**best_params)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1800, 7)]         0         
                                                                 
 conv1d (Conv1D)             (None, 600, 124)          4464      
                                                                 
 batch_normalization (BatchN  (None, 600, 124)         496       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 600, 124)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 200, 124)          77004     
                                                                 
 batch_normalization_1 (Batc  (None, 200, 124)         496       
 hNormalization)                                             

In [5]:
history = model.fit(x=data.X_train, y=data.y_train, validation_data=(data.X_test, data.y_test) , verbose=1, class_weight={0: 0.84, 1: 1.14},
                    epochs=2, batch_size=20)


Epoch 1/2
1283/1283 [==============================] - 52s 40ms/step - loss: 0.6654 - accuracy: 0.5860 - val_loss: 0.6833 - val_accuracy: 0.5991
Epoch 2/2
1283/1283 [==============================] - 47s 37ms/step - loss: 0.6655 - accuracy: 0.5905 - val_loss: 0.6829 - val_accuracy: 0.5991


In [6]:
time_string = time.strftime('%Y%m%d-%H%M%S')
model.save(str(config.MODEL_DIR_PATH.joinpath(f"{model_name}_{time_string}.h5")))


In [7]:
dump(history, str(config.MODEL_DIR_PATH.joinpath(f"histories/{model_name}_{time_string}.joblib")))

INFO:tensorflow:Assets written to: ram://9f633989-2299-4356-8b98-2b25ee0bf981/assets


['/home/tim/IM/data/models/60sec/histories/cnn_20220804-193427.joblib']

In [8]:
y_pred_test = model.predict(data.X_test)
y_pred_train = model.predict(data.X_train)

In [9]:
print(np.mean((y_pred_test > .5) == data.y_test))


0.5991067303249653


In [10]:
np.save(str(config.PREDICTION_DIR_PATH.joinpath(f"{model_name}_y_pred_test_{time.strftime('%Y%m%d-%H%M%S')}.npy")), y_pred_test)
np.save(str(config.PREDICTION_DIR_PATH.joinpath(f"{model_name}_y_pred_train_{time.strftime('%Y%m%d-%H%M%S')}.npy")), y_pred_train)


In [10]:
exit()
